## Problem 30: Find an Eulerian Path in a Graph

#### Eulerian Path Problem
Find an Eulerian path in a graph.

> Given: A directed graph that contains an Eulerian path, where the graph is given in the form of an adjacency list.

> Return: An Eulerian path in this graph.


<br>

#### Differences: 

**Eulerian cycle (term):**

Closed walk that uses every edge exactly once and ends where it started.
Condition (directed): every node has outdeg = indeg, and the edge-carrying part is connected.

**Eulerian path:**

Walk that uses every edge exactly once but start and end can differ.
Condition (directed): exactly one node has outdeg = indeg + 1 (start), exactly one has indeg = outdeg + 1 (end), all others balanced, and the edge-carrying part is connected.

**Why this matters:**

In assembly graphs (e.g., de Bruijn), a cycle reconstructs a circular/genome-like structure; a path reconstructs a linear sequence with defined start/end.

--

So, we can reuse our prev eulerian cycle function with some edits:

1. we find the start (out−in = +1) and end (−1).
2. we can then add a temporary edge from end -> start.
3. run eulerian cycle on this augmented graph
4. in the result cycle, split at that added edge to linearize it into an Eulerian path from start to end.

--

Example:

```
0 -> 2
1 -> 3
2 -> 1
3 -> 0,4
6 -> 3,7
7 -> 8
8 -> 9
9 -> 6
```

```
        ┌─────► 7 ─────► 8 ─────► 9 ─────► 6 ─────┐
        │                                         │
(start) 6                                         │
        │                                         │
        └─────► 3 ─────► 0 ─────► 2 ─────► 1 ─────┘
                                  │
                                  └────────► 3 ─────► 4 (end)
```



In [98]:
# ---- INPUT -----

text = """
0 -> 2
1 -> 3
2 -> 1
3 -> 0,4
6 -> 3,7
7 -> 8
8 -> 9
9 -> 6
"""

In [99]:
# pasring and building an adjacency dict (node -> list of neighbors)

adj = {}
nodes = set()

for line in text.strip().splitlines():
    left, right = [p.strip() for p in line.split("->")]
    u = left
    v_list = [v.strip() for v in right.split(",")] if right else []
    
    
    # tries to get val at u, if it doesnt exist itll initialize it to [], then appends v_list
    adj[u] = adj.get(u, []) + v_list
    nodes.add(u)
    for v in v_list:
        nodes.add(v)

# error fixed (important): making sure nodes with zero out-degree appear, so we dont get stuck as we cycle
for n in nodes:
    if n not in adj:
        adj[n] = []

# adj

In [100]:
# finding the start and end
# makign a dict, such that if there's a differnece between outdegrees and indegrees of a node
# then we can identify the start or end

# will hold (outdeg - indeg)
diff = {}  

for u, vs in adj.items():
    
    # add outdegrees
    diff[u] = diff.get(u, 0) + len(vs)      
    
    # subtract indegrees
    for v in vs:
        diff[v] = diff.get(v, 0) - 1    
        
#     print(diff)

    
start = next((n for n in diff if diff[n] == 1), None)
end = next((n for n in diff if diff[n] == -1), None)

print("\nstart, end =", (start,end))


start, end = ('6', '4')


In [101]:
# important assumption: graph is eulerian, meaning all nodes have 
# outgoing edges equal to incoming edges for a given node
# other: edges = nodes-1 for a closed walk

def eulerian_cycle(adj, start):

    # making a shallow copy 
    g = {u: list(vs) for u, vs in adj.items()}


#   print("start: ", start)
    stack, cycle = [start], []

    while stack:
        v = stack[-1]
        
        # if v exists, we pop from the adj_list, and append to stack
        if g[v]:
            w = g[v].pop()   # consume edge v->w
            stack.append(w)
            
        # if no vs under that g (all popped for that g), then we pop from stack and add to cycle
        else:
            popped = stack.pop()
            cycle.append(popped)
            
#         print("\nstack: ", stack)
#         print("cycle: ", cycle)
#         print(g)

    # reverse to correct order
    cycle.reverse()  
    
    return cycle



In [102]:
# add an edge from start to end
adj[end] = adj.get(end, []) + [start]

# eul path: get eul cycle with the correct start, then remove the final edge we augmented
cycle = eulerian_cycle(adj, start)[:-1]

path_str = "->".join(cycle)
print(path_str)

6->7->8->9->6->3->0->2->1->3->4
